In [1]:
%run ../Python_files/load_dicts.py
%run ../Python_files/util.py

# implement GLS method to estimate OD demand matrix
def GLS(x, A, L):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    A: path-link incidence matrix
    P: logit route choice probability matrix
    L: dimension of xi
    ----------------
    return: xi
    ----------------
    """
    K = np.size(x, 1)
    S = samp_cov(x)

    #print("rank of S is: \n")
    #print(matrix_rank(S))
    #print("sizes of S are: \n")
    #print(np.size(S, 0))
    #print(np.size(S, 1))

    inv_S = inv(S).real

    A_t = np.transpose(A)

    Q_ = np.dot(np.dot(A_t, inv_S), A)
    #Q = adj_PSD(Q_).real  # Ensure Q to be PSD
    Q = Q_

    #print("rank of Q is: \n")
    #print(matrix_rank(Q))
    #print("sizes of Q are: \n")
    #print(np.size(Q, 0))
    #print(np.size(Q, 1))

    b = sum([np.dot(np.dot(A_t, inv_S), x[:, k]) for k in range(K)])
    # print(b[0])
    # assert(1==2)

    model = Model("OD_matrix_estimation")

    xi = []
    for l in range(L):
        xi.append(model.addVar(name='xi_' + str(l)))

    model.update() 

    # Set objective: (K/2) xi' * Q * xi - b' * xi
    obj = 0
    for i in range(L):
        for j in range(L):
            obj += (1.0 / 2) * K * xi[i] * Q[i, j] * xi[j]
    for l in range(L):
        obj += - b[l] * xi[l]
    model.setObjective(obj)

    # Add constraint: xi >= 0
    for l in range(L):
        model.addConstr(xi[l] >= 0)
        #model.addConstr(xi[l] <= 5000)
    #fictitious_OD_list = zload('../temp_files/fictitious_OD_list')
    #for l in fictitious_OD_list:
        #model.addConstr(xi[l] == 0)
    model.update() 

    model.setParam('OutputFlag', False)
    model.optimize()

    xi_list = []
    for v in model.getVars():
        # print('%s %g' % (v.varName, v.x))
        xi_list.append(v.x)
    # print('Obj: %g' % obj.getValue())
    return xi_list

import numpy as np
from numpy.linalg import inv
import json

# load logit_route_choice_probability_matrix
P = zload('../temp_files/OD_pair_route_incidence_MA.pkz')
P = np.matrix(P)

# print(np.size(P,0), np.size(P,1))

# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix_MA.pkz')

# load link counts data
with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['AM_flow_minute'][minute_idx])

# print(len(link_day_minute_Jan_list))

x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

# print(np.size(x,0), np.size(x,1))
# print(x[:,:2])
# print(np.size(A,0), np.size(A,1))

L = np.size(P,1)  # dimension of xi

xi_list = GLS(x, A, L)
xi_dict = {}



In [4]:
L, len(xi_list), xi_list

(314,
 314,
 [199.7655663624385,
  40.50934408411093,
  27.119884164163615,
  182.0436247155953,
  92.08205196571613,
  36.10932003761841,
  52.45985905668817,
  30.307813616935764,
  41.16553146652821,
  32.149786616676515,
  22.84380556393342,
  70.69254525686576,
  37.29228752228872,
  40.39115587845548,
  115.96387570990723,
  33.016539910402116,
  47.602508534073465,
  64.21566865808065,
  75.32889713744687,
  61.70039170350297,
  38.16427113677818,
  109.55270338769135,
  79.97542771977528,
  81.38254605839222,
  68.38358790973706,
  67.59906939017662,
  40.50564924352956,
  35.830721996935246,
  96.17980776070931,
  70.81203443335723,
  95.70344078346032,
  46.330637687272294,
  48.653871477289556,
  38.791365818406376,
  30.56073744916913,
  51.51163841048769,
  51.511965552743575,
  176.64603670720427,
  91.21350577137721,
  38.49577204029313,
  168.45832747836315,
  115.92864872050185,
  41.41841732022956,
  37.04711199800832,
  66.90000996729461,
  29.104492413944598,
  33.9

In [9]:
model = Model("OD_matrix_and_route_choice_matrix")

L = np.size(P,0)  # dimension of lam

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))
    model.update()
    model.addConstr(lam[l] >= 0)
    
p = {}
for i in range(np.size(P,0)):
    for j in range(np.size(P,1)):
        p[(i,j)] = model.addVar(name='p_' + str(i) + ',' + str(j))
        model.update()
        model.addConstr(p[(i,j)] >= 0)
        if P[i,j] == 0:
            model.addConstr(p[(i,j)] == 0)

for i in range(np.size(P,0)):
    model.addConstr(sum([p[(i,j)] for j in range(np.size(P,1))]) == 1)

for idx in range(len(xi_list)):
    model.addConstr(sum([p[(l,idx)] * lam[l] for l in range(L)]) >= xi_list[idx])
    model.addConstr(sum([p[(l,idx)] * lam[l] for l in range(L)]) <= xi_list[idx])

model.update()

obj = 0
model.setObjective(obj)

model.update() 

model.setParam('OutputFlag', False)
model.optimize()

lam_list = []
for v in model.getVars():
    # print('%s %g' % (v.varName, v.x))
    if 'lam' in v.varName:
        lam_list.append(v.x)

GurobiError: Q matrix is not positive semi-definite (PSD)

In [2]:
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jan_weekend.py

In [3]:
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Apr_weekend.py

In [4]:
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Jul_weekend.py

In [5]:
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_AM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_MD.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_PM.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekday_NT.py
%run ../Python_files/OD_matrix_estimation_GLS_Oct_weekend.py